<a href="https://colab.research.google.com/github/DLPY/Classification_Session_1/blob/main/Churn_Random_Forest_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dataset used**

Churn Modelling: https://www.kaggle.com/shrutimechlearn/churn-modelling

# Step 1: Import the relelvant packages and read the dataset

In [ ]:
# import libraries 

import pandas as pd # Import Pandas for data manipulation using dataframes
import numpy as np # Import Numpy for data statistical analysis 
import matplotlib.pyplot as plt # Import matplotlib for data visualisation
%matplotlib inline
import seaborn as sns # Statistical data visualization

#Scikit-Learn otherwise known as sklearn is used for machine learning and has functionality for many types of classification models
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, auc, classification_report, confusion_matrix, ConfusionMatrixDisplay,
                             plot_roc_curve, PrecisionRecallDisplay, roc_auc_score, roc_curve)
from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import export_graphviz # Visualize Tree
from collections import OrderedDict
from collections import Counter #Python Counter is a container that will hold the count of each of the elements present in the container
from imblearn.under_sampling import RandomUnderSampler # provides tools when dealing with classification with imbalanced classes

from pprint import pprint


In [ ]:
# Load the data
# CSV is first read in from a github raw file another option is to import the notebook to your session storage by click on the file icon on left toolbar then importing csv
! wget https://raw.githubusercontent.com/DLPY/Classification_Session_1/master/Churn_Modelling.csv

In [ ]:
#reading the input file
df_data = pd.read_csv('Churn_Modelling.csv')
df_data.head()

**Churn Modeling Data Description**

This data set contains details of a bank's customers and the target variable is a binary variable reflecting the fact whether the customer left the bank (closed their account) or they continue to be a customer.

Here we have 13 feature columns and Exited is a target column.

Row Numbers: Row Numbers from 1 to 10000.

CustomerId: Unique Ids for bank customer identification.

Surname: Customer's last name.

CreditScore: Credit score of the customer.

Geography: The country from which the customer belongs(Germany/France/Spain).

Gender: Male or Female.

Age: Age of the customer.

Tenure: Number of years for which the customer has been with the bank.

Balance: Bank balance of the customer.

NumOfProducts: Number of bank products the customer is utilising.

HasCrCard: Binary Flag for whether the customer holds a credit card with the bank or not(0=No, 1=Yes).

IsActiveMember: Binary Flag for whether the customer is an active member with the bank or not(0=No, 1=Yes).

EstimatedSalary: Estimated salary of the customer in Euro.

Exited: Binary flag 1 if the customer closed account with bank and 0 if the customer is retained(0=No, 1=Yes).

# Step 2: Data Transformation

In [ ]:
#Removing columns that do not add value to the analysis
df_data = df_data.drop(['RowNumber', 'CustomerId','Surname'],axis=1)
df_data.head()

**Encoding the categorical variables - Change the text into numbers**

In [ ]:
#Convert the categorical values into numeric categorical labels so that this data can be used for modelling.
df_data['CountryCode'] = df_data['Geography'].astype('category').cat.codes
df_data['GenderCode'] = df_data['Gender'].astype('category').cat.codes

In [ ]:
df_data.head(5)

From the above, notice that:

The Geography and Gender have been converted to numeric values.

There are two new columns with these values: CountryCode and GenderCode.

In [ ]:
#droping the string columns as we have the respective numeric columns
df_data = df_data.drop(['Geography', 'Gender'],axis=1)

**Choosing predictor variables and target variable for performing Classification
Target and Source variables** 


Target Variable: Exited

Predictor Variables: CreditScore, CountryCode, GenderCode, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary

In [ ]:
X = df_data.drop(['Exited'],axis=1) #selecting all variables except "exited" as our predictor variables

y = df_data['Exited'] # selecting "Exited" column as our target variable

# Save this list of column values for later
columns_list = list(X.columns.values)

In [ ]:
X.head(5)

In [ ]:
y.head(5)

# Split dataset into the training and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

print('Training Data:', X_train.shape, y_train.shape)
print('Testing Data:', X_test.shape, y_test.shape)

# Train, Test and Predict using a Random Forest Classifier model

**Model 1 : Basic**

In [ ]:
# Create an object using RandomForestClassifier with default parameters
randomforest_classifier_m1 = RandomForestClassifier()
# Fit the classification model to the training set data.
randomforest_classifier_m1.fit(X_train, y_train)

In [ ]:
#Check how the model predicts training data
y_predict_train = randomforest_classifier_m1.predict(X_train)
#creating confusion matrix
cm = confusion_matrix(y_train, y_predict_train)

In [ ]:
#Plot confusion matrix
ConfusionMatrixDisplay.from_predictions(y_train, y_predict_train, cmap='RdPu_r')
plt.grid(False)

In [ ]:
#Classification report for training data 
print(classification_report(y_train, y_predict_train))

In [ ]:
#Check how the model predicts testing data
y_predict_m1 = randomforest_classifier_m1.predict(X_test)
#creating confusion matrix
cm_m1 = confusion_matrix(y_test, y_predict_m1)

In [ ]:
#Plot confusion matrix
ConfusionMatrixDisplay.from_predictions(y_test, y_predict_m1, cmap='RdPu_r')
plt.grid(False)

In [ ]:
#Classification report for testing data 
print(classification_report(y_test, y_predict_m1))

In [ ]:
print('Parameters currently in use:\n')
pprint(randomforest_classifier_m1.get_params())

In [ ]:
# Extract one of the trees from the model
tree = randomforest_classifier_m1.estimators_[6]

#defining features adn target variables
target = list(df_data['Exited'].unique())
feature_names = list(X.columns.values)

# Export as dot file
export_graphviz(tree, out_file='tree.dot',
feature_names =feature_names, class_names=['Left', 'Stayed'], rounded = True, proportion = False, precision = 2, filled = True, max_depth=3)
# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=90'])
# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

In [ ]:
# Extract one of the trees from the model
tree = randomforest_classifier_m1.estimators_[55]

#defining features adn target variables
target = list(df_data['Exited'].unique())
feature_names = list(X.columns.values)

# Export as dot file
export_graphviz(tree, out_file='tree.dot',
feature_names =feature_names, class_names=['Left', 'Stayed'], rounded = True, proportion = False, precision = 2, filled = True, max_depth=3)
# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=90'])
# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

**Model 2 : Using Undersampled Data**

Resampling the dataset due to class imbalance

In [ ]:
# summarize class distribution
print("Before undersampling: ", Counter(y_train))

# define undersampling strategy
undersample = RandomUnderSampler(sampling_strategy='majority')

# fit and apply the transform
X_train_under, y_train_under = undersample.fit_resample(X_train, y_train)

# summarize class distribution
print("After undersampling: ", Counter(y_train_under))

In [ ]:
# Create an object using RandomForestClassifier with default parameters
randomforest_classifier_m2 = RandomForestClassifier()
# Fit the classification model to the training set data.
randomforest_classifier_m2.fit(X_train_under, y_train_under)

In [ ]:
#Check how the model predicts testing data
y_predict_m2 = randomforest_classifier_m2.predict(X_test)
#creating confusion matrix
cm_m2 = confusion_matrix(y_test, y_predict_m2)

In [ ]:
#Plot confusion matrix
ConfusionMatrixDisplay.from_predictions(y_test, y_predict_m2, cmap='RdPu_r')
plt.grid(False)

In [ ]:
#Classification report for testing data 
print(classification_report(y_test, y_predict_m2))

In [ ]:
# NOTE: Setting the `warm_start` construction parameter to `True` disables
# support for parallelized ensembles but is necessary for tracking the OOB
# error trajectory during training.
ensemble_clfs = [
    (
        "RandomForestClassifier, max_features='sqrt'",
        RandomForestClassifier(
            warm_start=True,
            oob_score=True,
            max_features="sqrt",
            random_state=5,
        ),
    ),
    (
        "RandomForestClassifier, max_features='log2'",
        RandomForestClassifier(
            warm_start=True,
            max_features="log2",
            oob_score=True,
            random_state=5,
        ),
    ),
    (
        "RandomForestClassifier, max_features=None",
        RandomForestClassifier(
            warm_start=True,
            max_features=None,
            oob_score=True,
            random_state=5,
        ),
    ),
]

# Map a classifier name to a list of (<n_estimators>, <error rate>) pairs.
error_rate = OrderedDict((label, []) for label, _ in ensemble_clfs)

# Range of `n_estimators` values to explore.
min_estimators = 50
max_estimators = 700

for label, clf in ensemble_clfs:
    for i in range(min_estimators, max_estimators + 1, 50):
        clf.set_params(n_estimators=i)
        clf.fit(X, y)

        # Record the OOB error for each `n_estimators=i` setting.
        oob_error = 1 - clf.oob_score_
        error_rate[label].append((i, oob_error))

# Generate the "OOB error rate" vs. "n_estimators" plot.


In [ ]:
for label, clf_err in error_rate.items():
    xs, ys = zip(*clf_err)
    plt.plot(xs, ys, label=label)

plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate")
plt.legend(loc="upper right")
plt.show()

In [ ]:
pprint(error_rate)

**Model 3 : Using Best parameters found**

In [ ]:
# Create an object using DecisionTreeClassifier, setting a few parameters such as max depth.
randomforest_classifier_m3 = RandomForestClassifier(n_estimators = 600, criterion = 'gini',min_samples_split =7,min_samples_leaf =7,
                                                 max_features='log2',max_depth=None,bootstrap=True)
randomforest_classifier_m3.fit(X_train_under, y_train_under)
y_predict_m3 = randomforest_classifier_m3.predict(X_test)
cm_m3 = confusion_matrix(y_test, y_predict_m3)

In [ ]:
#Plot confusion matrix
ConfusionMatrixDisplay.from_predictions(y_test, y_predict_m3, cmap='RdPu_r')
plt.grid(False)

In [ ]:
#Classification report for testing data 
print(classification_report(y_test, y_predict_m3))

In [ ]:
feat_importances = pd.Series(randomforest_classifier_m3.feature_importances_, index=X.columns)
feat_importances.nlargest(10).sort_values().plot(kind='barh', title='Feature Importance')
plt.show()

In [ ]:
#Model comparison
print('Model 1 : Basic')
print(classification_report(y_test, y_predict_m1))
print('-----------------------------------------------------------------')
print('Model 2 : Using Undersampled Data')
print(classification_report(y_test, y_predict_m2))
print('-----------------------------------------------------------------')
print('Model 3 : Using Best parameters found ')
print(classification_report(y_test, y_predict_m3))
print('-----------------------------------------------------------------')

**Conclusion**

1. Model 1 has the highest accuracy compared to the other two models but that is misleading.

2. We notice that model 1 has a very low recall and high precision for Prediction of class 1 but that flips in the subsequent models.

3. The f1-score is the best for our last model which gives us some confidence that the steps applied have improved the performance of the model

4. The support for Class 1 prediction is low (624) compared to Class 0 prediction (2376) which again shows the data still an imbalance issue and hence other sampling techniques can be applied to achieve better models.



